In [1]:
import pandas as pd
import os

Get text file

In [5]:

swed_file = open(os.getcwd()+"\\dat410_europarl\\europarl-v7.sv-en.lc.sv", mode='rt', encoding='utf-8')
en_file = open(os.getcwd()+"\\dat410_europarl\\europarl-v7.sv-en.lc.en", mode='rt', encoding='utf-8')

# read all texts
swed_text = swed_file.read()
en_text = en_file.read()
# close the file
swed_file.close()
en_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Sina\\Interview\\dat410_europarl\\europarl-v7.sv-en.lc.sv'

In [42]:
swed_sentences = swed_text.strip().replace(".","").replace(",","").replace("&quot;","").split('\n')
en_sentences = en_text.strip().split('\n')


In [46]:
swed_sentences[7]

'skulle det vara möjligt för er  fru talman  att skriva ett brev till den srilankesiska presidenten i vilket parlamentets beklagande uttrycks över hans och de övriga brutala dödsfallen i sri lanka och uppmanar henne att göra allt som står i hennes makt för att få en fredlig lösning på en mycket komplicerad situation ?'

In [49]:
from collections import Counter

The 10 most frequent words in Swedish

In [65]:
swed_clean_text = swed_text.replace(".","").replace(",","").replace("&quot;","").replace("\n","").replace("  ","").strip()
swed_words = swed_clean_text.split(' ')
swed_counter_Words = Counter(swed_words)
swed_counter_Words.most_common(10)

[('att', 8896),
 ('och', 6110),
 ('i', 5715),
 ('det', 5347),
 ('för', 4632),
 ('som', 4425),
 ('av', 3961),
 ('är', 3680),
 ('en', 3570),
 ('vi', 3096)]

The 10 most frequent words in english

In [66]:
en_swed_text = en_text.replace(".","").replace(",","").replace("&quot;","").replace("\n","").replace("  ","").strip()
en_words = en_swed_text.split(' ')
en_counter_Words = Counter(en_words)
en_counter_Words.most_common(10)

[('the', 18235),
 ('of', 9119),
 ('to', 8514),
 ('and', 5910),
 ('in', 5497),
 ('is', 4139),
 ('a', 4083),
 ('that', 3920),
 ('this', 2924),
 ('for', 2657)]

In [89]:
def get_probability(word,counter_Words):
    value_word = counter_Words.get(word)
    value_word= value_word if value_word is not None else 0
    total_Count=sum(counter_Words.values())
    return value_word/float(total_Count) * 100


what is the probability that it is speaker?

In [73]:
get_probability('speaker',en_counter_Words)

0.0032880405744206884

What is the probability that it is zebra?

In [90]:
get_probability('zebra',en_counter_Words)

0.0